# Primitivas Geométricas y Transformaciones

In [2]:
import numpy as np
import cv2

In [3]:
def muestra_imagen(imagen):
    cv2.imshow("Canvas base", imagen)
    cv2.waitKey(0)

In [4]:
w = 250
h = 250

imagen_base = np.zeros((w, h))
muestra_imagen(imagen_base)

QFontDatabase: Cannot find font directory /home/ruben/Documentos/Codigos/Python/VisionArtificial/VisionArtificial/venv/lib/python3.11/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/ruben/Documentos/Codigos/Python/VisionArtificial/VisionArtificial/venv/lib/python3.11/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/ruben/Documentos/Codigos/Python/VisionArtificial/VisionArtificial/venv/lib/python3.11/site-packages/cv2/qt/fonts.
Note that Qt no longer ships fonts. Deploy some (from https://dejavu-fonts.github.io/ for example) or switch to fontconfig.
QFontDatabase: Cannot find font directory /home/ruben/Documentos/Codigos/Python/VisionArtificial/VisionArtificial/venv/lib/python3.11/site-pac

In [6]:
def obtener_homogeneas(coordenadas_2D, w):
    return np.hstack([coordenadas_2D, w])

def obtener_2D(coordenadas_homogeneas):
    return cv2.convertPointsFromHomogeneous(coordenadas_homogeneas)

In [7]:
# Slicing
imagen_base[int(w/2-25):int(w/2+25), int(h/2-25):int(h/2+25)] = 100
muestra_imagen(imagen_base)

x = 100
y = 100
escala = 1

# Punto 2D
p = [x, y]
# Punto 2D Coordenadas Homogeneas
p_h = np.array([[x, y, escala]])

print(obtener_homogeneas(p, escala))
print(obtener_2D(p_h))

[100 100   1]
[[[100. 100.]]]


# Transformaciones Euclideanas

In [14]:
class TransformacionesEuclideanas:
    @staticmethod
    def rotacion(angulo, s=1):
        return np.array([
            [s*np.cos(angulo), -s*np.sin(angulo), 0],
            [s*np.sin(angulo), s*np.cos(angulo), 0],
            [0, 0, 1]
        ])
    
    @staticmethod
    def traslado(x,y):
        return np.array([
            [1, 0, x],
            [0, 1, y],
            [0, 0, 1]
        ])
    
    @staticmethod
    def reflexion(eje='x'):
        if eje == 'x':  # Refleja sobre eje Y
            return np.array([[-1, 0, 0],
                             [0, 1, 0],
                             [0, 0, 1]])
        elif eje == 'y':  # Refleja sobre eje X
            return np.array([[1, 0, 0],
                             [0, -1, 0],
                             [0, 0, 1]])
        elif eje == 'origen':  # Refleja sobre origen
            return np.array([[-1, 0, 0],
                             [0, -1, 0],
                             [0, 0, 1]])

In [ ]:
ruta = "ovalo.png"
imagen_base = cv2.imread(ruta, cv2.IMREAD_GRAYSCALE)
h,w = imagen_base.shape


In [ ]:
imagen_transformada = np.zeros((h, w), dtype=imagen_base.dtype)
centro_x, centro_y = int(round(h/2)), int(round(w/2))

R = TransformacionesEuclideanas.rotacion(-np.pi/8, 0.5)
T1 = TransformacionesEuclideanas.traslado(-centro_x, -centro_y)
T2 = TransformacionesEuclideanas.traslado(centro_x, centro_y)

#T = traslado2 @ rotacion @ traslado @ es producto matricial
T = T2 @ R @ T1

M_inv = np.linalg.inv(T)
for x in range(w):
    for y in range(h):
        p_destino_h = np.array([x, y, 1]) # Vector Columna - Trnspuesto de [y, x, 1]
        p_origen_h = M_inv @ p_destino_h

        x_origen = p_origen_h[0] / p_origen_h[2]
        y_origen = p_origen_h[1] / p_origen_h[2]

        #Si está dentro de la imagen original
        if 0 <= x_origen < w and 0 <= y_origen < h:
        # Interpolación por vecino más cercano
            y_idx = int(np.floor(y_origen))
            x_idx = int(np.floor(x_origen))
            imagen_transformada[y, x] = imagen_base[y_idx, x_idx]
            
muestra_imagen(imagen_transformada)

250 250 (250, 250) 1 0
250 250 (250, 250) 0 2
250 250 (250, 250) 3 0
250 250 (250, 250) 2 2
250 250 (250, 250) 1 4
250 250 (250, 250) 0 6
250 250 (250, 250) 0 8
250 250 (250, 250) 5 1
250 250 (250, 250) 4 3
250 250 (250, 250) 3 5
250 250 (250, 250) 2 7
250 250 (250, 250) 1 9
250 250 (250, 250) 1 10
250 250 (250, 250) 0 12
250 250 (250, 250) 7 0
250 250 (250, 250) 6 2
250 250 (250, 250) 6 4
250 250 (250, 250) 5 6
250 250 (250, 250) 4 8
250 250 (250, 250) 3 9
250 250 (250, 250) 3 11
250 250 (250, 250) 2 13
250 250 (250, 250) 1 15
250 250 (250, 250) 0 17
250 250 (250, 250) 0 19
250 250 (250, 250) 9 1
250 250 (250, 250) 8 3
250 250 (250, 250) 7 5
250 250 (250, 250) 7 6
250 250 (250, 250) 6 8
250 250 (250, 250) 5 10
250 250 (250, 250) 4 12
250 250 (250, 250) 4 14
250 250 (250, 250) 3 16
250 250 (250, 250) 2 18
250 250 (250, 250) 1 19
250 250 (250, 250) 1 21
250 250 (250, 250) 0 23
250 250 (250, 250) 12 0
250 250 (250, 250) 11 2
250 250 (250, 250) 10 4
250 250 (250, 250) 9 5
250 250 (250, 25

In [ ]:
"""
Tarea:
Dibujar tres imagenes en Paint (ajustar el tamaño de la imagen). Guardar en png
Aplicar 4 transformaciones euclidenanas.
"""